In [ ]:

def stations_clusters_on_map(clustered_stations: pd.DataFrame):
    # Create a map centered at the mean coordinates of all stations
    center_lat = clustered_stations['latitude'].mean()
    center_lon = clustered_stations['longitude'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add markers for each station, colored by cluster
    colors = ['#%06x' % np.random.randint(0, 0xFFFFFF) for _ in range(clustered_stations["cluster"].nunique())]
    for _, row in clustered_stations.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=colors[int(row["cluster"])] if row["cluster"] != -1 else 'gray',
            fill=True,
            popup=f"Cluster {row["cluster"]}"
        ).add_to(m)

    """# Add markers for cluster centers
    for i, center in cluster_centers.iterrows():
        folium.Marker(
            location=[center['latitude'], center['longitude']],
            icon=folium.Icon(color='black', icon='info-sign'),
            popup=f'Cluster Center {i}'
        ).add_to(m)"""
    display(m)

stations_clusters_on_map(clustered_stations_dbscan)

In [ ]:
from matplotlib import colors as mcolors
from folium.plugins import HeatMap

# SUMMED DEMAND HEATMAP (cell 19)

# sum total demand per cluster across the dataset
cluster_total = df_demand.groupby('cluster')['demand'].sum().reset_index()

cluster_centers = clustered_stations_kmeans.groupby('cluster')[['latitude', 'longitude']].mean()
center_lat = cluster_centers['latitude'].mean()
center_lon = cluster_centers['longitude'].mean()

# merge with cluster centers (cluster_centers index corresponds to cluster id)
cluster_centers_idx = cluster_centers.reset_index().rename(columns={'index': 'cluster'})
cluster_total = cluster_total.merge(cluster_centers_idx, on='cluster', how='left').dropna(subset=['latitude','longitude'])

# normalize sizes and colors
max_d = cluster_total['demand'].max()
cluster_total['radius'] = (cluster_total['demand'] / max_d) * 40 + 5  # radius in pixels

cmap = plt.cm.viridis
norm = plt.Normalize(vmin=cluster_total['demand'].min(), vmax=max_d)

# create folium map centered on existing center
m_demand = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# add circle markers sized & colored by total demand
for _, r in cluster_total.iterrows():
    color = mcolors.to_hex(cmap(norm(r['demand'])))
    folium.CircleMarker(
        location=[r['latitude'], r['longitude']],
        radius=float(r['radius']),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Cluster {int(r['cluster'])} — Demand: {int(r['demand'])}"
    ).add_to(m_demand)

# add HeatMap layer (weights proportional to demand)
heat_data = [[r['latitude'], r['longitude'], r['demand']/max_d] for _, r in cluster_total.iterrows()]
HeatMap(heat_data, radius=25, max_zoom=13).add_to(m_demand)

m_demand